## Deploy your pre-trained keras model to AWS

In [9]:
pip install tensorflow==2.6

     |████████████████████████████████| 458.3 MB 8.3 kB/s             
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.2
    Uninstalling tensorflow-2.6.2:
      Successfully uninstalled tensorflow-2.6.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-serving-api 2.1.0 requires tensorflow~=2.1.0, but you have tensorflow 2.6.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install sagemaker==2.86.2

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install mediapipe

  Using cached mediapipe-0.8.3-cp36-cp36m-manylinux2014_x86_64.whl (67.0 MB)
  Using cached numpy-1.19.3-cp36-cp36m-manylinux2010_x86_64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-serving-api 2.1.0 requires tensorflow~=2.1.0, but you have tensorflow 2.6.0 which is incompatible.
tensorflow-cpu 2.1.3 requires gast==0.2.2, but you have gast 0.4.0 which is incompatible.
tensorflow-cpu 2.1.3 requires h5py<=2.10.0, but you have h5py 3.1.0 which is incompatible.
tensorflow-cpu 2.1.3 requires keras-preprocessing==1.1.0, but you have keras-preprocessing 1.1.2 which is incompatible.
tensorflow-cpu 2.1.3 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.19.3 which is incompatible.
tensorflow

In [3]:
#pip install tensorflow==1.15.5

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


     |████████████████████████████████| 67.0 MB 71.6 MB/s            
     |████████████████████████████████| 14.9 MB 49.0 MB/s            
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
Note: you may need to restart the kernel to use updated packages.


In [40]:
import numpy as np

# if your model is saved as only a .h5 file
MODEL_LOCATION ='sampleModel'


## 2. Load Your Model
Simply run the cell below; the model will be loaded based on how you defined the above

In [41]:
if MODEL_LOCATION!='': #if your model is saved as a .h5 file only
    from tensorflow.keras.preprocessing.image import img_to_array
    from tensorflow.keras.models import load_model
    model = load_model(MODEL_LOCATION, compile = False)#ad the model
    print("loaded model from MODEL_LOCATION")

loaded model from MODEL_LOCATION


In [42]:
import tensorflow as tf
tf.__version__

'2.6.0'

## 3. Convert the Keras Model to the format AWS wants
- Converts to a Protobuff file
- Saves it in a certain aws file structure
- Tarballs this file and zips it

In [43]:
def convert_to_aws(loaded_model):
    """
    given a pre-trained keras model, this function converts it to a TF protobuf format
    and saves it in the file structure which aws expects
    """  
    from tensorflow.python.saved_model import builder
    from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
    from tensorflow.python.saved_model import tag_constants
    import tensorflow as tf
    import os
    import shutil
    if tf.executing_eagerly():
       tf.compat.v1.disable_eager_execution()
    dirpath = 'export'
    if os.path.exists(dirpath) and os.path.isdir(dirpath):
        shutil.rmtree(dirpath)
    
    # This is the file structure which AWS expects. Cannot be changed. 
    model_version = '1'
    export_dir = 'export/Servo/' + model_version
    
    # Build the Protocol Buffer SavedModel at 'export_dir'
    builder = builder.SavedModelBuilder(export_dir)
    
    # Create prediction signature to be used by TensorFlow Serving Predict API
    signature = predict_signature_def(
        inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})
    
    session = tf.compat.v1.Session()
    init_op = tf.compat.v1.global_variables_initializer()
    session.run(init_op)
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=session, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save() 
    
    #create a tarball/tar file and zip it
    import tarfile
    with tarfile.open('model_deploy.tar.gz', mode='w:gz') as archive:
        archive.add('export', recursive=True)
        
convert_to_aws(model)

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


## 4. Move the tarball (tar.gz) to S3

In [44]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model_deploy.tar.gz', key_prefix='model')

# View details of the uploaded bucket
print(f"Bucket name is: {sagemaker_session.default_bucket()}")

Bucket name is: sagemaker-us-east-1-852069333125


This is the name of the bucket which SageMaker made in S3

In [45]:
# where did it upload to?
print("Bucket name is:")
sagemaker_session.default_bucket()

Bucket name is:


'sagemaker-us-east-1-852069333125'

## 5. Create a SageMaker Model
First, create an empty train.py file (TensorFlowModel expects this at its 'entry point', but can be empty)

In [46]:
!touch train.py #create an empty python file

In [47]:
import boto3, re
import tensorflow as tf
from sagemaker import get_execution_role

# the (default) IAM role you created when creating this notebook
role = get_execution_role()

# Create a Sagemaker model (see AWS console>SageMaker>Models)
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model_deploy.tar.gz',
                                  role = role,
                                  framework_version = tf.__version__,
                                  entry_point = 'train.py')

## 6a) Host the SageMaker model and
## 6b) Create an Endpoint to access the model 

Ignore the message `update_endpoint is a no-op in sagemaker>=2`

In [48]:
deployement_instance_type = "ml.m4.xlarge"
from sagemaker.serializers import CSVSerializer
predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type=deployement_instance_type)
endpoint = predictor.endpoint

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


--------!

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [49]:
endpoint

'tensorflow-inference-2022-05-18-17-22-29-121'

## Success! You have deployed a keras model into AWS
# ---------------------
### 7. Confirm its working correctly by making a prediction
Now, we want to use our endpoint/model. Create a predictor which uses the endpoint

In [50]:
import cv2
import mediapipe as mp
import numpy as np
import copy
import itertools

In [51]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []
    
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        
        landmark_point.append([landmark_x, landmark_y])

    return landmark_point


def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    
    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list))

    
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

def get_key(val):
    for key, value in labels_dict.items():
        if val == value:
            return key
    return "key doesn't exist"

In [52]:

mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils
labels_dict = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,'Z':25,'space':26,'del':27,'nothing':28}

In [54]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.serializers import CSVSerializer
import json


endpoint = 'tensorflow-inference-2022-05-18-17-22-29-121' #get endpoint name from SageMaker > endpoints

predictor=sagemaker.predictor.Predictor(
    endpoint_name='tensorflow-inference-2022-05-18-17-22-29-121',
    sagemaker_session=sagemaker.Session(),
    serializer=CSVSerializer())

#success, image = cap.read()
image = cv2.imread("W_test.jpg", cv2.IMREAD_COLOR)
imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = hands.process(imgRGB)
predictions=""
if results.multi_hand_landmarks:
    for hand_landmarks, handedness in zip(results.multi_hand_landmarks,results.multi_handedness):
        landmark_list = calc_landmark_list(imgRGB, hand_landmarks)
        pre_processed_landmark_list = pre_process_landmark(landmark_list)
        data = np.array([pre_processed_landmark_list], dtype=np.float32)
        predictions = predictor.predict(data)
        my_json = predictions.decode("utf-8").replace("'", '"')
        jsonData = json.loads(my_json)
        classes_x=np.argmax(np.squeeze(jsonData["predictions"]))
        print("\nLabel")
        print(get_key(classes_x))
        print("\n")
        cv2.putText(image, get_key(classes_x),(10,60), cv2.FONT_HERSHEY_PLAIN,3, (255,0,255),4)




Label
space




In [39]:
classes_x

53

## Cleanup!

else you will incur extra charges

https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html

- Stop Notebook
- delete endpoints
- delete models
- delete S3 bucket
- delete cloudwatch groups
